In [ ]:
! pip install --quiet fuzzywuzzy
! pip install --quiet nltk
! pip install --quiet diskcache
! pip install --quiet python-Levenshtein
! pip install --quiet lightgbm
! pip install -e 'git://github.com/nandanrao/embed-software.git#egg=embed_software'

In [1]:
%load_ext autoreload 
%autoreload 2

import pandas as pd
import numpy as np
import re
from validation.data import indeed_test_data, dot_train_data

In [6]:
SAMPLE_SIZE = 999999999
SOC_LEVEL = 6

In [13]:
# X_train, y_train = dot_train_data(SOC_LEVEL)
X_test, y_test, ids = indeed_test_data('us-everything.csv', SAMPLE_SIZE, SOC_LEVEL)

In [62]:
df = pd.read_csv('us-everything.csv')

In [64]:
dot_dict = pd.read_excel('soc-definitions-2000.xlsx', skiprows=6)
dot_dict['soc'] = get_soc_n(dot_dict['2000 SOC code'], 6)

found = pd.DataFrame({'y': y_test}).merge(dot_dict, how='left', left_on='y', right_on='soc')
found.index = y_test.index

In [65]:
labelled = (pd
            .concat([df.iloc[found.index], found], 1)
            .pipe(lambda df: df.assign(description = df.content.str.replace('\t', '. ')))
            .drop([ 'y', 'soc', 'content'], 1))

In [69]:
labelled.to_csv('indeed-us-titles-matched.csv', index=False)